In [2]:
# gmailer init


In [1]:
import datetime as dt

import yaml

import requests
import smtplib
import ssl
import mimetypes
from email.message import EmailMessage

import pandas as pd



## Load Configurations

In [16]:
## password and config
config = "../config.yaml"
config = yaml.safe_load(open(config))

def parse_config(config: dict):
    """
    Parse the configuration yaml.

    Parameters
    -------
    config (dict{any}): Dictionary of congiruations.

    Returns
    -------
    credentials (dict(any)): Dictionary of credentials.
    """
    credentials = config["credentials"]
    ...
    return credentials

credentials = parse_config(config)

# Methods

In [161]:
def get_csv_as_list(filepath: str) -> list:
    """
    Get list using a path to a csv.

    Parameters
    -------
    filepath (str): filepath. 

    Returns
    -------
    ([str]): List of values.
    
    """
    import pandas as pd
    recipients = pd.read_csv(recipients_csv_path, header=None)
    return recipients[0].values.tolist()


#### Potential Class 0
class Clearbit():
    """A class for working with Clearbit API."""
    def get_name(self, response: requests.models.Response) -> (str, str):
        """
        Parse response for HTTP request.

        Parameters
        -------
        response (requests.models.Response): Http response with person data.

        Returns
        -------
        name (str, str): Tuple of first and last name. 
        """
        if not isinstance(response, requests.models.Response):
            print("Not a request.")
            return 
        name = (None, None)
        data = response.json()

        person = data["person"]
        first_name = person["name"]["givenName"]
        last_name = person["name"]["familyName"]
        name = (first_name, last_name)
        return name


    def get_names_from_email_list(self, recipients_list:[str], username=None, password=None, api_key=None):
        """
        Given a list of recipients, use Clearbit to retreive their names. Other data may be retreieved but at a later stage. 
        The username is the api_key from Clearbit. Read their docs for more info.

        Parameters
        -------
        username (str): Optional. Username.
        password (str): Optional. Password.
        api_key (str): API key.

        Returns
        -------
        names {}: Dict of emails to names.
        """
        names = {}
        # NOTE: may want to batch this in the future or too many requests will be attempted too quickly.
        for email in recipients_list:
            url = f"https://person.clearbit.com/v2/combined/find?email=:{email}"
            clearbit_response = get_response(url, username=username, password=password, api_key=api_key)
            names[email] = self.get_name(clearbit_response)
        return names
#####


def get_response(url: str, username=None, password=None, api_key=None):
    """
    Get a response from API using HTTP.

    Parameters
    -------
    url (str): Url for API request.
    username (str): Optional. Username.
    password (str): Optional. Password.
    api_key (str): API key.

    Returns
    -------
    api_response (requests.models.Response): Response.
    """
    api_response = None
    api_response = requests.get(url, auth=(username, password))
    return api_response


# Email class

In [174]:
def send_mail(
    sender: str, recipients: list, subject:str, 
    password: str, 
    clearbit_user=None,
    attachment_path=None,
    body=None, body_path=None, body_config=None
):
    """
    Send an email via SMTP. Recommended body is provided as HTML formatted text. Provide body or body_path and body_config but not both. 

    Parameters
    -------
    sender (str): Sender as a string.
    recipients ([str]): List of recipients. 
    subject (str): Email subject line.
    password (str): Gmail password as string.
    clearbit_user (str): Optional. Clearbit username. 
    attachment_path (str): Optional. Filepath to attachment. 
    body (str): Optional. Optional. Email body as string.
    body_path (str): Optional. Email body path to be parsed using the body config. Recommended to use HTML formatting.
    body_config (str): Optional. Email config for the body including variables that can be quickly parsed and replaced.
    
    Returns
    -------
    None
    """
    def add_attachment(email: EmailMessage, attachment_path: str) -> EmailMessage:
        """
        Given original email message. May or may not include an attachment already.

        Parameters:
        -------
        email (EmailMessage): EmailMessage.
        attachment_path (str): Filepath to attachment.

        Returns
        -------
        email (EmailMessage): Mail message with new attachment.
        """
        # Attachments
        with open(attachment_path, "rb") as fp:
            data = fp.read()

        # guess encoding
        ctype, encoding = mimetypes.guess_type(attachment_path)
        if ctype is None or encoding is not None:
            # No guess could be made, or the file is encoded (compressed), so
            # use a generic bag-of-bits type.
            ctype = "application/octet-stream"
        maintype, subtype = ctype.split("/", 1)

        email.add_attachment(data, maintype=maintype, subtype=subtype)
        print(f"Successfully attached: {attachment_path}")
        return email


    def build_text(text_path: str, text_vars=None) -> str:
        """
        Using passed-along dictionary of variable names to values, fill in the 
        text file located at text_path. May be passed on no variables to fill, in which case the text body is returned as-is.
        Ignores case (case-insensitive). 

        Parameters
        -------
        text_path (str): Path to text file.
        text_vars (dict): Dictionary of variables. Optional

        Returns
        -------
        query (str): Filled-in text body by value using the text_vars dictionary.
        """
        import regex as re

        DEFAULT_FILL_IN = ""

        # Get the text.
        with open(text_path, "r+") as f:
            text = f.read() 
        f.close()
        if text_vars == None:
            return text

        # Extract variables from the text.
        variables_in_text = re.findall("\{(.*?)\}", text, flags=re.IGNORECASE)

        # do a replacement: each time call local_vars...
        for var in variables_in_text:
            replace_me = "\{" + var + "\}"
            replace_with = str(text_vars.get(var,DEFAULT_FILL_IN))
            text = re.sub(replace_me, replace_with, text)
        return text

    if body != None and (body_path != None or body_config != None):
        print("Provide body or body_path and body_config but not both.")
        return

    email = EmailMessage()
    email["Sender"] = sender
    email["Recipients"] = ", ".join(recipients)
    email["Subject"] = subject

    if body != None:
        email.set_content(body)

    if attachment_path != None:
        email = add_attachment(email, attachment_path)

    # Email failsafe.
    confirm_send = input(f"Are you sure you want to send to recipients? (Y/N) \n\n {recipients}\n")

    # Get names using emails with Clearbit.
    cb = Clearbit()
    names_from_emails = cb.get_names_from_email_list(recipients, username=clearbit_user)

    # Do email sending.
    if confirm_send.lower() == "y":
        context = ssl.create_default_context()
        print("Sending email...")
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp_server:
            smtp_server.login(sender, password)
            for to in recipients:
                 # Value 0 indicates first name, 1 is last name.
                body_config["addressee"] = names_from_emails[to][0]
                body = build_text(body_path, body_config)
                email.set_content(body, subtype="html")
                smtp_server.sendmail(sender, ["jaime.meriz13@gmail.com"], email.as_string())
                
        print("Message sent!")
    else:
        print("Message NOT sent!")

    return email



# Testing the password trigger sender.

In [175]:
# password
password = credentials["gmail"]["app_password"]
clearbit_api = credentials["clearbit"]["api_key"]

sender = "jaime.meriz13@gmail.com"

# get senders from file path
# recipient path should be: list of recipients. Single recipient.
recipients_csv_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/email_recipients.csv"
recipients = get_csv_as_list(recipients_csv_path)

# subject 
subject = "DEV" + "-" "Jmail" + "-" + str(dt.date.today())


# 
body_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/Email Outreaches/generic_1/body.txt"
body = "body\nyours\nDr. Lopez"

body_config_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/Email Outreaches/generic_1/body.yaml"
body_config = yaml.safe_load(open(body_config_path))

email = send_mail(
    sender, recipients, subject, password, clearbit_user=clearbit_api, body_path=body_path, body_config=body_config,
)


Sending email...
Message sent!


# Email Body Formatting

In [137]:
filepath = "/Users/jaimemerizalde/Desktop/JOBS 2023/Email Outreaches/generic_1/body.yaml"
body_config = yaml.safe_load(open(filepath))

# filepath = 
body_config

{'company': 'Garner Health',
 'job_industry': 'health care',
 'role': 'Data Science engineering',
 'job_post_origin': 'Flexjobs.com',
 'tools_used': 'Python, Snowflake, Buildkite, Pandas, Heroku, Swagger, Git, Pycharm CE, Heroku, and MS Azure',
 'web_linkedIn': 'https://www.linkedin.com/in/jaimemeriz/',
 'web_git': 'https://github.com/jlopez8',
 'web_1': 'https://scatoolsuite.gitbook.io/sca-tool-suite/',
 'my_first': 'Jaime',
 'my_last': 'Lopez-Merizalde',
 'my_email': 'jaime.meriz13@gmail.com',
 'my_phone': '+1 402-707-9444'}

In [57]:
body_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/Email Outreaches/generic_1/body.txt"
# text_vars = body_config
body = build_text(body_path, body_config)
email.set_content(body, subtype="html")


In [58]:
sender = "jaime.meriz13@gmail.com"
recipients = ["jaime.meriz13@gmail.com"]
password = credentials["gmail"]["app_password"]

context = ssl.create_default_context()

with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp_server:
    smtp_server.login(sender, password)
    smtp_server.sendmail(sender, recipients, email.as_string())
    print("Message sent!")


Message sent!


# Parsing clearbit

In [108]:


username = credentials["clearbit"]["api_key"]

result = Clearbit().get_names_from_email_list(recipients, username=username)

name = result["marco.starger@getgarner.com"][0]

name




'Marco'

In [93]:
cb = Clearbit()

cb.get_names_from_email_list